## Машинное обучение, ВМК МГУ

## Практическое задание 2

### Общая информация
Дата выдачи: 9 октября 2019

Максимальная оценка: 10 баллов + 1 бонусный балл

Мягкий дедлайн: 23:59MSK 23 октября (за каждый день просрочки снимается 1 балл)

Жесткий дедлайн: 23:59MSK 30 октября.

### О задании

В этом задании вы:
- Познакомитесь с методом решения задачи регрессии на основе метода ближайших соседей.
- Реализуете алгоритм kNN для задачи регрессии.
- Изучите методы работы с категориальными и текстовыми переменными.


### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.


### Формат сдачи
Для сдачи задания переименуйте получившийся файл *.ipynb в соответствии со следующим форматом: homework-practice-02-Username.ipynb, где Username — ваша фамилия и имя на латинице именно в таком порядке (например, homework-practice-02-ivanov.ipynb).

Далее отправьте этот файл на anytask в соответсвующий раздел.

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_absolute_error

In [2]:
def RMSE(predictions, targets):
    n = len(predictions)
    rmse = np.linalg.norm(predictions - targets) / np.sqrt(n)
    return rmse

Все эксперименты в этой лабораторной работе предлагается проводить на данных соревнования New York City Airbnb Open Data: https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data#AB_NYC_2019.csv

В данной задаче предлагается предсказать цену на съем квартиры в зависимости от её параметров.

In [3]:
data = pd.read_csv('AB_NYC_2019.csv')
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [4]:
data.shape

(48895, 16)

In [5]:
# число значений у признаков
for col_name in data.columns:
    print(col_name, len(data[col_name].unique()))

id 48895
name 47906
host_id 37457
host_name 11453
neighbourhood_group 5
neighbourhood 221
latitude 19048
longitude 14718
room_type 3
price 674
minimum_nights 109
number_of_reviews 394
last_review 1765
reviews_per_month 938
calculated_host_listings_count 47
availability_365 366


In [6]:
data.isna().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

Как видите, в данных есть пропуски. Не забудьте обработать их

In [7]:
# Ваш код здесь (ノ°∀°)ノ⌒･*:.｡. .｡.:*･゜ﾟ･*☆

In [8]:
data['reviews_per_month'].fillna(data[0, inplace=True)
data['name'].fillna("No_name", inplace=True)
data['host_name'].fillna("No_name", inplace=True)
data['last_review'].fillna("No_reviews", inplace=True)

Разобъем данные на обучение и контроль.

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['price']), data[['price']],
                                                    test_size=0.3, random_state=241)

## Часть 1: Алгоритм kNN в задаче регрессии

<b> 1.1 (1.5 балла) </b>
Реализуйте класс `KNNRegressor`, который используя метод k ближайших соседей решает задачу регрессии. Для решение данной задачи, необходимо найти $N_k$ - k соседей, и после использовать значения их целевых переменных для предсказания:
\begin{align}
y = \frac{1}{k}\sum_{n \in N_k}w_n y_n,
\end{align}

где $w_n$ - вес каждого соседа. 

При этом `KNNRegressor` может работать в 2 режимах:
 - $uniform$ - ближайшие соседи учитываются с одинаковыми весами.
 - $distance$ - вес ближайших соседей зависит от расстояния
 
Сигнатуру методов при желании можно менять.

In [209]:
from typing import Union, Callable, Iterable, Optional
from sklearn.neighbors import NearestNeighbors


def make_weights(input_arr: np.array):
    all_dist = np.sum(input_arr)
    if all_dist == 0:
        return input_arr + (1/input_arr.__len__())
    return input_arr / all_dist

class KNNRegressor:
    def __init__(self, n_neighbors: int,
                 metric: Union[str, Callable],
                 mode: str = 'uniform',
                 algorithm: str = 'brute') -> None:
        """
        params:
            n_neighbors: number of neighbors
            metric: metric to use for distance computation
            mode: 'uniform' or 'distance'
            'uniform' - all points in each neighborhood are weighted equally
            'distance' - weight points by the inverse of their distance
        """
        
        self.metric = metric
        self.ne = NearestNeighbors(n_neighbors=n_neighbors, algorithm=algorithm, metric=metric, n_jobs = -1)
        
        if mode != 'uniform' and mode != 'distance':
            raise TypeError
        self.mode = mode
        self.k = n_neighbors
        
    def changeCatMetric(self, metric: Callable):
        
        if ('overlap' not in str(metric)):
            raise TypeError
            
        self.metric = metric
        
    def changeMode(self, mode: str):
        if mode != 'uniform' and mode != 'distance':
            raise TypeError
        self.mode = mode

    def fit(self, X: np.array, y: np.array) -> None:
        """
            X: data
            y: labels
        """
        self.train_target = y
        
        if ('overlap' in str(self.metric)):
            # здесь оно подаёт нормально, index = элемент
            if X.shape.__len__() == 1:
                self.train_set = X[:, None]
                print(X[:, None].shape, self.train_set.shape)
            else:
                self.train_set = X

        else:
            self.ne.fit(X, y)
        
        
        
    def kneighbors(self, X: np.array, k = 0, return_distance: bool = True):
        
        if ('overlap' in str(self.metric)):
            """categorial metric area, we'd have to calculate distance for each of attributes separately
               как я решил строить входные данные для этого метода - по строкам расположены данные столбцов входных данных
               метрики будут возвращать соседей по строкам для каждого элемента по каждому из признаков
               потом мы просуммируем эти элементы"""
            
            if k == 0:
                k = self.k

            if X.shape.__len__() == 1:
                test = X[None, :]
            else:
                test = X

                
            # Пока что попробуем просто mae по всем элементам
            dist_by_ind = np.zeros(self.train_set.__len__() * test.__len__()).reshape(test.__len__(), -1)

            for index in range(self.train_set.shape[1]):
                dist_by_ind += self.metric(self.train_set.T[index], test.T[index]) ** 2
                # по идее, оно должно найти расстояния по 1 из кат. признаков
                
                
            dist_by_ind = np.sqrt(dist_by_ind)
            indexes = np.argsort(dist_by_ind, axis=1)[:, :k]
            
            if return_distance:
                distances = np.empty(indexes.shape)
                for enum, item in enumerate(indexes):
                    distances[enum] = dist_by_ind[enum, indexes[enum]]
                return distances, indexes
            
            return indexes
            
        else:
            if k == 0:
                return self.ne.kneighbors(X, return_distance=return_distance)
            return self.ne.kneighbors(X,n_neighbors=k, return_distance=return_distance)

    def predict(self, X: np.array, n_neighbors: Optional[int] = None) -> np.array:
        """
            X: data
            n_neighbors: number of neighbors
        """ 
        
        # checking for which k we have to make our neighbors
        if n_neighbors is None:
            k = self.k
        else:
            if n_neighbors > 0:
                k = n_neighbors
            else:
                k = self.k

        test_target = np.empty(X.__len__())
        
        if self.mode == 'uniform':
            nearest = self.kneighbors(X, k, return_distance=False)
            print(nearest.shape)
            for enum in range(X.__len__()):
                target = 0
                
                for numb in range(nearest[enum].__len__()):
                    target += self.train_target[nearest[enum][numb]] / k
                test_target[enum] = target

        else:
            dist, nearest = self.kneighbors(X, k, return_distance=True)

            for enum in range(X.__len__()):
                target = 0
                weights = make_weights(dist[enum])
                
                for numb in range(nearest[enum].__len__()):
                    target += self.train_target[nearest[enum, numb]] * weights[numb]

                test_target[enum] = target
                
        return test_target

## Часть 2: Категориальные признаки

<b>2.1 (1 балл)</b>
Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на [третьем семинаре](https://github.com/mmp-mmro-team/mmp_mmro_fall_2019/blob/master/lecture-notes/Sem03_knn.pdf). Не забудьте, что KNNRegressor должен уметь работать с этими функциями расстояния. Как вариант, можно реализовать метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html).

In [210]:
def getComonness(array: np.array):
    """Процент каждого объекта классфикации в выборке + объект на соотв. позиции"""
    uniques, counts = np.unique(array, return_counts=True)
    return counts / np.sum(counts), uniques

def getFFunc(array: np.array):
    """Шанс встретить 2 случайных одинаковых объектов в выборке, отсортированный в словаре"""
    uniques, counts = np.unique(array, return_counts=True)
    first_func = (counts * (counts - 1)) / np.sum(counts) / (np.sum(counts) - 1)
    
    return first_func

In [988]:
from math import log
def overlap(x, z):
    """0 при совпадении, 1 иначе"""
    array = np.empty([x.__len__(), z.__len__()])
    
    for enum in range(array.__len__()):
        array[enum] = z != x[enum]
    
    return array.T

def flattened_overlap(x, z):
    answer = np.zeros(len(x) * len(z)).reshape(len(x), len(z))
  
    values, counts = np.unique(x, return_counts=True)
    freq = counts / len(x)
    
    for i, elem in enumerate(x):
        
        index = np.where(elem == values)[0][0]
        q = np.where(freq <= freq[index])[0]
        unique_sum = np.sum(counts[q] * (counts[q] - 1) / (len(x) * (len(x) - 1)))
        answer[i] = (elem != z).astype(int) + (elem == z).astype(int) * unique_sum
        
    return answer.T

def log_overlap(x, z):
    answer = np.zeros(len(x) * len(z)).reshape(len(x), len(z))
    
    freq_x = np.empty(x.__len__())
    freq_z = np.empty(z.__len__())
    
    for i in range(len(x)):
        freq_x[i] = len(np.where(x[i] == x)[0])
        
    for i in range(len(z)):
        freq_z[i] = len(np.where(z[i] == z)[0])
    
    log_x = np.log(freq_x + 1)
    log_z = np.log(freq_z + 1)
    
    for i, elem in enumerate(x):
        
        answer[i] = (elem != z).astype(int) * log_x[i] * log_z
    return answer.T

<b>2.2 (1 балл)</b> Найдите все категориальные признаки в данных. Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Качество измеряйте с помощью RMSE.

Какая функция расстояния оказалась лучшей? Почему?

In [959]:
target = np.array(data['price'])
cat_data = np.array((data['room_type'],
    data['name'],
                     data['host_name'],
                     data['neighbourhood_group'],
                     data['neighbourhood'],
                     data['last_review'])).T

In [960]:
train_data, test_data, train_target, test_target = train_test_split(cat_data, target,
                                                    test_size=0.3, random_state=241)

In [989]:
knn = KNNRegressor(10, flattened_overlap, 'uniform', 'brute')

In [990]:
knn.fit(train_data, train_target)

In [31]:
%time res_overlap = knn.predict(test_data)

(14669, 10)
CPU times: user 2min 52s, sys: 1min 56s, total: 4min 48s
Wall time: 5min 37s


In [918]:
knn.changeCatMetric(flattened_overlap)

In [931]:
%time res_flat = knn.predict(test_data)

(14669, 10)
CPU times: user 5min 17s, sys: 2min 6s, total: 7min 23s
Wall time: 8min 16s


In [932]:
RMSE(test_target, res_flat)

211.16893448825135

In [991]:
knn.changeCatMetric(log_overlap)

In [992]:
%time res_log = knn.predict(test_data)

(14669, 10)
CPU times: user 6min 38s, sys: 3min 10s, total: 9min 48s
Wall time: 13min 44s


In [993]:
RMSE(test_target, res_log)

240.73489949844412

In [299]:
print("RMSE overlap: ", RMSE(test_target, res_overlap))
print("RMSE flat: ", RMSE(test_target, res_flat))
print("RMSE log: ", RMSE(test_target, res_log))

RMSE overlap:  218.080625930482
RMSE flat:  211.16893448825135
RMSE log:  240.73489949844412


In [285]:
print("MAE overlap: ", mean_absolute_error(test_target, res_overlap))
print("MAE flat: ", mean_absolute_error(test_target, res_flat))
print("MAE log: ", mean_absolute_error(test_target, res_log))

MAE overlap:  75.19959779126047
MAE flat:  74.30134296816415
MAE log:  86.96225373236075


In [76]:
train_target = np.array(y_train).T[0]
test_target = np.array(y_test).T[0]

train_name, test_name = np.array(X_train['name']).T, np.array(X_test['name']).T
train_host_name, test_host_name = np.array(X_train['host_name']).T, np.array(X_test['host_name']).T
train_nhg, test_nhg = np.array(X_train['neighbourhood_group']).T, np.array(X_test["neighbourhood_group"]).T
train_nh, test_nh = np.array(X_train['neighbourhood']).T, np.array(X_test['neighbourhood']).T
train_rt, test_rt = np.array(X_train['room_type']).T, np.array(X_test['room_type']).T
train_date, test_date = np.array(X_train['last_review']).T, np.array(X_test['last_review']).T

In [77]:
knn = KNNRegressor(10, flattened_overlap, "distance", "brute")

In [78]:
knn.fit(train_name, train_target)
res_name = knn.predict(test_name)

(34226, 1) (34226, 1)


In [79]:
knn.fit(train_host_name, train_target)
res_host_name = knn.predict(test_host_name)

(34226, 1) (34226, 1)


In [80]:
knn.fit(train_nhg, train_target)
res_nhg = knn.predict(test_nhg)

(34226, 1) (34226, 1)


In [81]:
knn.fit(train_nh, train_target)
res_nh = knn.predict(test_nh)

(34226, 1) (34226, 1)


In [82]:
knn.fit(train_rt, train_target)
res_rt = knn.predict(test_rt)

(34226, 1) (34226, 1)


In [83]:
knn.fit(train_date, train_target)
res_date = knn.predict(test_date)

(34226, 1) (34226, 1)


In [300]:
print("Results of single classified parameters methods: ")
print("name: ", RMSE(test_target, res_name), mean_absolute_error(test_target, res_name))
print("host_name: ", RMSE(test_target, res_host_name), mean_absolute_error(test_target, res_host_name))
print("neighbourhood_group: ", RMSE(test_target, res_nhg), mean_absolute_error(test_target, res_nhg))
print("neigbhourhood: ", RMSE(test_target, res_nh), mean_absolute_error(test_target, res_nh))
print("room_type: ", RMSE(test_target, res_rt), mean_absolute_error(test_target, res_rt))
print("latest_review: ", RMSE(test_target, res_date), mean_absolute_error(test_target, res_date))

Results of single classified parameters methods: 
name:  234.29299364006755 134.73003396348065
host_name:  239.80449092681968 120.32290615517809
neighbourhood_group:  219.09318516763017 79.2914922625946
neigbhourhood:  221.22132548657578 88.36691371560482
room_type:  212.10555627933817 68.13100415842933
latest_review:  234.85839988655533 101.29124264163715


In [85]:
train_combo = np.c_[train_nhg, train_rt]
test_combo = np.c_[test_nhg, test_rt]

In [87]:
knn.fit(train_combo, train_target)
res_combo = knn.predict(test_combo)

In [88]:
mean_squared_error(test_target, res_combo), mean_absolute_error(test_target, res_combo)

(61396.95097582071, 118.37272479378281)

<b>2.3 (1 балл) бонус</b> Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какого удалось достичь уровня качества?

In [91]:
knn.fit(train_rt, train_target)
for enum in range(1, 11):
    cur_res = knn.predict(test_rt, n_neighbors=enum)
    print("For k = ", enum, ": ", RMSE(test_target, cur_res), mean_absolute_error(test_target, cur_res))
    

(34226, 1) (34226, 1)
For k =  1 :  74858.74572227146 163.6885268252778
For k =  2 :  47504.99139341468 91.89620969391233
For k =  3 :  45591.20704281895 77.40266321267072
For k =  4 :  46064.06531631331 82.6260481287068
For k =  5 :  45513.98614220465 76.3460631263208
For k =  6 :  45145.73344013453 72.64044356579635
For k =  7 :  45128.60591334495 69.75965836603916
For k =  8 :  44961.670379499286 68.65047890108391
For k =  9 :  45094.377561987196 67.86193863097537
For k =  10 :  44988.76700456746 68.13100415842933


<b>2.4 (2.5 балла)</b> Отойдем ненадолго от задачи регрессии и перейдём к задаче классификации: будем определять, являеться ли квартира дорогой $(target = 1)$ или дешевой $(target = 0)$. Будем считать дорогими квариры, цена которых выше среднего.

In [52]:
data['target'] = (data.price > data.price.mean()).astype(int)

Реализуйте счетчики, которые заменят категориальные признаки на вещественные.

А именно, для каждого категориального признака $f_j(x)$ необходимо сделать следующее:
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
\begin{align}
counts_j(c) = \sum_{i=1}^l [f_j(x_i) = c]
\end{align}
2. Число `successes` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
\begin{align}
successes_j(c) = \sum_{i=1}^l[f_j(x_i) = c][y_i = +1].
\end{align}
3. Сглаженное отношение двух предыдущих величин:
\begin{align}
p_j(c) = \frac{successes_j(c) + a}{counts_j(c) + b},
\end{align}

где $a$ и $b$ - априорные счетчики (например, a = 1, b = 2).

In [46]:
def counters(x):
    uniq, counts = np.unique(x, return_counts=True)
    
    result = np.empty(x.shape)
    
    for enum, item in enumerate(uniq):
        positions = np.argwhere(x == item)
        result[positions] = counts[enum]
    
    return result

def success_count(x, y):
    # x - cat value
    # y - target
    sub = x + y.astype(str)
    uniq, counts = np.unique(sub, return_counts=True)
    
    result = np.zeros(x.shape)
    
    for enum, item in enumerate(uniq):
        if item[-1] == '0':
            continue
        
        positions = np.argwhere(sub == item) 
        result[positions] = counts[enum]
        
        pos_neg = np.argwhere(sub == (item[:-1] + '0'))
        result[pos_neg] = counts[enum]
        
    return result

def comparison(x, y, a, b):
    count = counters(x)
    success = success_count(x, y)
    
    return (success + a) / (count + b)

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `successes` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанный по всей обучающей выборке. Реализуйте и такой вариант. Достаточно взять $n = 3$.

In [47]:
def kfold(n, n_folds):

    index_list = np.arange(n)

    result_list = []
    part = 0
    each_len = int(n / n_folds)

    while part < n_folds - 1:
        test_subset = np.array(index_list[part * each_len: (part + 1) * each_len])
        train_subset = np.array([x for x in index_list if x not in test_subset])
        result_list.append((train_subset, test_subset))
        part += 1

    test_subset = np.array(index_list[part * each_len:])
    train_subset = np.array([x for x in index_list if x not in test_subset])

    result_list.append((train_subset, test_subset))

    return result_list

def fold_counters(x):
    """
    params:
        x: value on categorical feature for N objects
    returns: vector of length N
    """
    
    cv = kfold(x.__len__(), 5)
    
    result = np.empty(x.__len__())
    
    for train_labels, test_labels in cv:
        count = counters(x[train_labels])
        
        extra = np.empty(test_labels.__len__())
        
        for enum, item in enumerate(x[test_labels]):
            pos = np.where(x[train_labels] == item)[0]
            if pos.__len__() == 0:
                extra[enum] = 0
            else:
                extra[enum] = count[pos[0]]
        
        result[test_labels] = extra
    
    return result

def fold_success_count(x, y):
    
    cv = kfold(x.__len__(), 5)
    
    result = np.empty(x.__len__())
        
    for train_labels, test_labels in cv:
        
        success = success_count(x[train_labels], y[train_labels])
        
        extra = np.empty(test_labels.__len__())
        
        for enum in range(test_labels.__len__()):
            cur_val = x[test_labels][enum]
            pos = np.where(x[train_labels] == cur_val)[0]
            
            if pos.__len__() == 0:
                extra[enum] = 0
            else:
                extra[enum] = success[pos[0]]
                
        result[test_labels] = extra
        
    return result

def fold_combine(x, y, a, b):
    count = fold_counters(x)
    success = fold_success_count(x, y)
    return (success + a) / (count + b)

Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

In [48]:
from sklearn.metrics import roc_auc_score

In [54]:
def changeCols(df, cols, folds = False):
    
    if folds:
        desired_func = fold_combine
    else:
        desired_func = comparison
        
    for item in cols:
        df[item + 'new'] = desired_func(np.array(df[item]), np.array(df['target']), 0, 0)
        
    return df

def transformAccordingTo(train_df, test_df, cols, folds=False):
    # окей... нужно трансформировать столбцы test так же, как столбцы train
    train_df = changeCols(train_df, cols, folds)
    for item in cols:
        
        replacement = np.empty(test_df.__len__())
        target_col = np.array(train_df[item])
        target_replacement = np.array(train_df[item+'new'])
        
        for enum, value in enumerate(test_df[item]):
            
            pos = np.argwhere(target_col == value).T[0]
            if pos.__len__() == 0:
                replacement[enum] = 0
            else:
                replacement[enum] = target_replacement[pos[0]]
                
        test_df[item] = replacement
        train_df[item] = train_df[item + 'new']
        train_df.drop(columns={item + 'new'}, inplace=True)

In [55]:
train_data, test_data, train_target, test_target = train_test_split(pd.DataFrame.copy(data),
                                                                    np.array(pd.DataFrame.copy(data['target'])),
                                                                    test_size=0.3, random_state=241)

In [56]:
transformAccordingTo(train_data, test_data, ['name', 'host_name', 'neighbourhood_group', 'neighbourhood', 'room_type', 'last_review'])

In [57]:
train_data.drop(columns={'id', 'host_id','target', 'price'}, inplace=True)
test_data.drop(columns={'id', 'host_id', 'target', 'price'}, inplace=True)

In [58]:
train_data = (train_data - train_data.mean()) / train_data.std()
test_data = (test_data - test_data.mean()) / test_data.std()

In [62]:
from sklearn.neighbors import KNeighborsClassifier
knn_cos = KNeighborsClassifier(n_neighbors=10, metric='cosine', algorithm='brute', weights='distance')
knn_euc = KNeighborsClassifier(n_neighbors=10, metric='euclidean', algorithm='brute', weights='distance')

In [63]:
knn_cos.fit(np.array(train_data), train_target)
knn_euc.fit(np.array(train_data), train_target)

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='distance')

In [64]:
result_cos_no_folds = knn_cos.predict(np.array(test_data))
result_euc_no_folds = knn_euc.predict(np.array(test_data))

In [66]:
print("Roc-auc euc: ", roc_auc_score(test_target, result_euc_no_folds))
print("Roc-auc cos: ", roc_auc_score(test_target, result_cos_no_folds))

Roc-auc euc:  0.5447485593462442
Roc-auc cos:  0.5492123814181694


In [67]:
train_data, test_data, train_target, test_target = train_test_split(pd.DataFrame.copy(data),
                                                                    np.array(pd.DataFrame.copy(data['target'])),
                                                                    test_size=0.3, random_state=241)

In [68]:
transformAccordingTo(train_data,
                     test_data,
                     ['name', 'host_name', 'neighbourhood_group', 'neighbourhood', 'room_type', 'last_review'],
                     True)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in true_divide


In [69]:
train_data.drop(columns={'id', 'host_id','target', 'price'}, inplace=True)
test_data.drop(columns={'id', 'host_id', 'target', 'price'}, inplace=True)
train_data = (train_data - train_data.mean()) / train_data.std()
test_data = (test_data - test_data.mean()) / test_data.std()
train_data.fillna(0, inplace = True)
test_data.fillna(0, inplace = True)

In [70]:
knn_cos.fit(np.array(train_data), train_target)
knn_euc.fit(np.array(train_data), train_target)

KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='euclidean',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='distance')

In [71]:
result_cos = knn_cos.predict(np.array(test_data))
result_euc = knn_euc.predict(np.array(test_data))

In [72]:
print("folded Roc-auc euc: ", roc_auc_score(test_target, result_euc))
print("folded Roc-auc cos: ", roc_auc_score(test_target, result_cos))

folded Roc-auc euc:  0.7707152214738934
folded Roc-auc cos:  0.7665220124515243


<b>2.5 (1 балл)</b> Вернемся к задаче регрессии. Утверждается, что для задачи регрессии можно также сделать преобразование категориальных признаков в действительные числа. Для этого достаточно для каждого значения признака $f_j$ вычислить:
\begin{align}
p_j(c) = g(T_i | f_j(x_i) = c),
\end{align}

где $T_i$ - значения целевой переменной объекта $x_i$. Функция $g$ - среднее (mean) или среднеквадратичное отклонение (std).

Закодируйте категориальные признаки обоими способами и найдите значение RMSE. Используйте евклидову метрику для поиска ближайших соседей. Для какой функции $g$ значение RMSE лучше? Почему?

In [ ]:
train_data, test_data, train_target, test_target = train_test_split(pd.DataFrame.copy(data.drop(columns={'target'})),
                                                                    np.array(pd.DataFrame.copy(data['price'])),
                                                                    test_size=0.3, random_state=241)

In [ ]:
def Transforming_df(df, cols):
    
    for item in cols:
        
        helper = df.groupby(item, as_index=False)['price'].mean()
        
        array = np.empty(df.__len__())
        
        for enum, element in enumerate(df[item]):
            
            array[enum] = np.array(helper['price'][np.argwhere(helper[item] == element)[0]])[0]
            
        df[item+"new"] = array

def Transform_test_df(train_df, test_df, cols):
    
    Transforming_df(train_df, cols)

    for item in cols:
        
        replacement = np.empty(test_df.__len__())
        target_col = np.array(train_df[item])
        target_replacement = np.array(train_df[item+'new'])
        
        for enum, value in enumerate(test_df[item]):
            
            pos = np.argwhere(target_col == value).T[0]
            if pos.__len__() == 0:
                replacement[enum] = 0
            else:
                replacement[enum] = target_replacement[pos[0]]
                
        test_df[item] = replacement
        train_df[item] = train_df[item + 'new']
        train_df.drop(columns={item + 'new'}, inplace=True)

In [ ]:
Transform_test_df(train_data, test_data, 
                 ['name', 'host_name', 'neighbourhood_group', 'neighbourhood', 'room_type', 'last_review'])

In [73]:
train_data.head(2)

,name,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
39319,0.0,-0.130386,-0.680178,-0.991473,-1.698327,-0.379113,-1.041794,-0.251955,-0.254864,-0.509586,1.581089,-0.125950,0.816011
22796,0.0,0.036191,1.086473,1.005422,0.091509,-0.631690,0.955213,-0.251955,-0.187906,-0.862348,-0.321471,-0.186474,-0.858349


In [ ]:
train_data.drop(columns={'id', 'host_id', 'price'}, inplace=True)
test_data.drop(columns={'id', 'host_id', 'price'}, inplace=True)

In [ ]:
train_data = (train_data - train_data.mean()) / train_data.std()
test_data = (test_data - test_data.mean()) / test_data.std()

In [ ]:
knn_reg = KNNRegressor(n_neighbors=10, metric='euclidean', mode='uniform', algorithm = 'brute')

In [ ]:
knn_reg.fit(np.array(train_data), train_target)

In [ ]:
res_reg = knn.predict(np.array(test_data))

In [ ]:
mean_squared_error(test_target, res_reg)

## Часть 3: Текстовые признаки

<b>3.1 (2 балла)</b> Перейдем от категориальным признаков к текстовым. Рассмотрим 2 способа преобразования текста в действительные числа:
- Мешок слов (Bag of Words)
- TF-IDF

[Здесь](https://scikit-learn.org/stable/modules/feature_extraction.html) вы можете прочитать про их применение в Питоне.

Сравните оба способа на задаче регресси. Какую лучше метрику использовать: евклидову или косинусную меру? Постройте графики зависимости качества решения задачи от способа преобразования, метрики и количества соседей. Мера качества - RMSE.

Объясните полученные результаты.

Перед преобразованием не забудьте уменьшить размер словаря. Например, это можно сделать за счет приведения всех слов к одному регистру и удаления [стопслов](https://en.wikipedia.org/wiki/Stop_words) (артиклей, предлогов, союзов).

In [11]:
# Приведу пример использования tf-idf своими руками
from sklearn.feature_extraction import stop_words
stop_list = ['herself', 'other', 'has', 'twenty', 'seeming', 'last', 'been', 'many', 'namely', 'per', 'you', 'whenever', 'in', 'hence', 'often', 'sincere', 'everywhere', 'yours', 'ours', 'find', 'done', 'somehow', 'formerly', 'alone', 'whose', 'system', 'seems', 'together', 'over', 'himself', 'elsewhere', 'already', 'call', 'ltd', 'sixty', 'mostly', 'still', 'or', 'yourself', 'thence', 'nobody', 'themselves', 'thick', 'de', 'have', 'no', 'upon', 'these', 'anyhow', 'un', 'couldnt', 'thin', 'detail', 'be', 'much', 'while', 'fifteen', 'co', 'from', 'now', 'herein', 'besides', 'can', 'might', 'itself', 'latter', 'could', 'wherever', 'none', 'anywhere', 'whereafter', 'else', 'thereupon', 'had', 'mine', 'anyone', 'take', 'nine', 'something', 'neither', 'nothing', 'twelve', 'again', 'side', 'are', 'beyond', 'than', 'back', 'hereafter', 'less', 'do', 'against', 'beside', 'your', 'me', 'three', 'were', 'forty', 'meanwhile', 'not', 'whom', 'etc', 'fifty', 'whole', 'several', 'afterwards', 'almost', 'during', 'con', 'thus', 'for', 'move', 'bottom', 'everyone', 'though', 'becoming', 'who', 'becomes', 'until', 'mill', 'bill', 'above', 'between', 'whereby', 'was', 'would', 'they', 'us', 'nor', 'never', 'there', 'cannot', 'may', 'former', 'what', 'please', 'top', 'further', 'empty', 'part', 'which', 'up', 'describe', 'whereupon', 'but', 'via', 'very', 'at', 'ever', 'therefore', 'with', 'latterly', 'within', 'full', 'a', 'get', 'along', 'i', 'next', 'either', 'hers', 'of', 'found', 'whether', 'should', 'two', 'although', 'down', 'to', 'front', 'eg', 'yet', 'thru', 'anything', 'on', 'since', 'perhaps', 'such', 'hereby', 'under', 'where', 'somewhere', 'any', 'one', 'behind', 'being', 'six', 'give', 'thereafter', 'every', 'my', 'five', 'hasnt', 'without', 'amongst', 'four', 'first', 'is', 'our', 'he', 'here', 'someone', 'it', 'by', 'when', 'became', 'more', 'most', 'own', 'third', 'others', 'keep', 'why', 'nevertheless', 'must', 'will', 'across', 'as', 'eight', 'after', 'whence', 'amoungst', 'sometimes', 'then', 'into', 'even', 'so', 'also', 'if', 'whereas', 'thereby', 'her', 'among', 're', 'throughout', 'seem', 'hundred', 'amount', 'therein', 'cant', 'fill', 'serious', 'and', 'that', 'anyway', 'well', 'myself', 'everything', 'ourselves', 'toward', 'indeed', 'about', 'we', 'made', 'see', 'noone', 'how', 'except', 'inc', 'his', 'otherwise', 'sometime', 'go', 'onto', 'those', 'yourselves', 'off', 'eleven', 'them', 'both', 'whither', 'ten', 'him', 'beforehand', 'ie', 'out', 'few', 'same', 'all', 'she', 'whatever', 'before', 'due', 'its', 'their', 'towards', 'am', 'always', 'the', 'seemed', 'only', 'because', 'cry', 'enough', 'fire', 'nowhere', 'show', 'an', 'least', 'too', 'once', 'put', 'interest', 'through', 'below', 'another', 'each', 'wherein', 'some', 'become', 'name', 'whoever', 'around', 'rather', 'this', 'hereupon', 'moreover', 'however']
stop_list.append("")

In [12]:
transf_array = np.array(data['name'])
def mass_replace(string, deletions):
    
    for item in deletions:
        string = string.replace(item, "")
        
    return string

In [13]:
for enum, item in enumerate(transf_array):
    transf_array[enum] = item.lower()
    transf_array[enum] = transf_array[enum].replace("  ", " ")
    transf_array[enum] = transf_array[enum].replace("/", " ")
    transf_array[enum] = transf_array[enum].replace("-", " ")
    transf_array[enum] = transf_array[enum].replace("...", " ")

for enum in range(transf_array.__len__()):
    
    transf_array[enum] = mass_replace(transf_array[enum], ["!", ",", "&", ".", "'", ":", "*"])

In [14]:
uniq_dict_host = {}
first_col = []

for item in transf_array:
    line = item.split(" ")
    line_kept = [x for x in line if x not in stop_list]
    first_col.append(line_kept)
    
    for part in line_kept:
        line_cur = []
        if part not in line_cur:
            line_cur.append(part)
            if part in uniq_dict_host:
                uniq_dict_host[part] += 1
            else:
                uniq_dict_host[part] = 1

In [15]:
sorted_dict = [(k, uniq_dict_host[k]) for k in sorted(uniq_dict_host, key=uniq_dict_host.get, reverse=True)]
words_list = []
numbers_list = []

from math import log

for item in sorted_dict:
    words_list.append(item[0])
    numbers_list.append(item[1])
    
idf = np.empty(words_list.__len__())
for enum in range(words_list.__len__()):
    idf[enum] = log(48995 / numbers_list[enum])


NameError: name 'log' is not defined

In [483]:
j = 0
for item in sorted_dict:
    if item[1] < 100:
        print(item[0])
        j += 1

b
artistic
simple
rental
c
lic
hudson
town
circle
cobble
unit
level
madison
serene
sunnyside
just
fidi
wburg
1st
mid
45
station
easy
urban
d
#2
ac
pvt
fantastic
pre
welcome
italy
deluxe
north
broadway
financial
live
lincoln
dryer
war
beauty
flatiron
murray
flatbush
2bed
bldg
vibrant
15min
#1
guests
exposed
sunset
museum
budget
term
escape
water
homey
j
wyndham
red
breakfast
corner
ditmas
4br
casa
story
adorable
good
fun
fee
8
wonderful
waterfront
massive
victorian
amenities
contemporary
twin
spectacular
2nd
relaxing
★
washer
service
n
m
colorful
price
rise
ferry
express
centrally
fabulous
en
nest
stops
enjoy
green
authentic
century
paradise
true
middle
subways
people
fl
quite
lots
deal
alcove
tree
cheap
industrial
professionals
stadium
vacation
creative
boerum
ground
sized
tranquil
5th
couch
la
club
medical
yankee
20min
love
hills
hidden
boutique
travelers
riverside
vintage
triplex
john
luxe
stuyvesant
calm
eclectic
design
lit
116
office
incredible
12
dumbo
chill
palace
walking
morning

morris
blues
nest;
60
lived
2e
amenties
area20
lighted
(luxury
things
1fl
shady
army
beachside
10minute
parachute
(rm3)
(rm2)
bd(c)
willimsburg
marine
vanilla
crownheights
$2800
37
sweetspot
twist
15minutes
adventurer
antiques
blend
38th
(single
large+amazing
greyhound
masterpiece
(rear)
flor
entrancefull
dd
temple
bed”
(ground
hugedeck
beautifu
roomprivate
chris
(jefferson
fairy
catering
4bedroom
minuets
solar
transp
camping
harlem)
rooming
kg
quiete
(queen)
(available
bedroomm
900mb
doorstep
sqre
activist
killer
2br+
oceanside
healthy
heater
tonight
hirise
2people
️
winged
sprawling
conscious
vivo
twinxl
#forreal
★backyard
harrigan
ambient
(and
weekender
80th
124
onebedroom
inspiration
002
doctors
cocoon
bahia
brazil
☝
(flexible)
used
rooftop)
hollis
smokers
pacific
mh
❤️❤️❤️
taaffe
45th
studiocozygymdoorman
secs
flatiron)
jfk+mall
sm
room20
102
shays
rh
hill+
window)
(large
gct
(studio)
4bedprivaterenovated
(between
waterview
brooklynny
unicorn
furnish
10minlga
30minjfk
235
bkft
hea

(
)
5
sunnyclean
法拉盛summer家(b)closed
diggs
mothership
曼哈顿景色豪华公寓
bedfd
loftmorgan
northshore
amazin
intimes
georgeous
stories
west★
@ferryhuge
privaterenovated
entranceroom
creating
social
khimanis
harvey
refugees
bathroomentrance
partitioned
varrazano
promenadenight
cat)
简单的四房一厅两卫生间位于北上远离开辆走路四分钟到地铁站交通便利
simple
bigroom
wac+hbo
term+++)
bachelors
loisaida
gazebo
otheroom
pea
"borough
liner"
3333
2bathcentralparkcolumbiajacuzzifireplace
2fireplcsgardenbbqwshr
dryerjacuztubbig
(2r)
bars6
$3500
manhattan(high
rise)
jfkone
(huge)
core
special
cozy

•superhost•
30mn
(train
assembly
amphitheat
mels
nycclose
4000sqft
nonprofit
☀️private
manhattan☀️
rosalee
stewart
jfklaguardia
a1long
cuny
a2long
a4long
hgv
plymouth
bitchin
curran
citycollege
modernized
greatly
bedroom
midtown
@ferryloft2bdrm
tag
shang
ri
glittering
seperate
maujer
negotiable
touring
uesideal
flowery
19minutes
apartent
mem
1mth+
styvesant
lcd
apt+sleeps
nurturing
area★★明るいお部屋★
levelclose
☆the
$0
"little
10mn
和缘特色浪漫房
upstate
71w


tb4)
judge
yourselfg
launch
3beds2br2bath
干净卫生的环境房欢迎订房
(d3)
sustainable
photoshoot
economic
4gr
“l”
affordable
warmclean
rochesters
★single
accommodation★
(2bath)
21min
neighborhoodprivateroom
voila
energies
(pool
stmarks
germanias
highclass
te
place]
zzz
steinwaythe
中城西豪华客厅
(12x12
chic+charming
(harlem
find)
harlemites
jfk30
carlga
bellissima
stanza
privata

with
pilots
attendant
freesia
(4a)
(2t)
citysolo
catalina’s
room

pictures)
cortelyou
alojamiento
betsaida
demetrio’s
rv
paisa(camarote#2abajo)
(camarote#1
arriba)
comfy’s
#6131
#6132
“mi
casa”
“my
home”
only2
sq1br
marcelo’s

close
studio~w
unit~prime
spaciousquitebeautiful
0)
vacate
★affordable
hell
kitchen★
(camarote#4
parte
baja)
comfortablespacious
settings
(semi
private)
nicea
1718公馆套房
lcazand
lacz
3br~w
曼哈顿108街转租
subway+city
mosholu
remote
modernspacious
alex
addtn
blowup
monthlylux
dollhouse
mondern
timbuktu
suits
bamako
fartsy
clubhouse
commode
locationcozy
★warm
private2br
z★
instagram
★columbus
circle★full
loft~5
beds@b

In [484]:
print(j)

10607


In [235]:
words = np.array(words_list)
numbers = np.array(numbers_list)

In [236]:
tf_idf = np.zeros(first_col.__len__())

for enum, item in enumerate(first_col):
    uniq, count = np.unique(np.array(item), return_counts=True)
    tf = count / np.sum(count)
    for enumer, word in enumerate(uniq):
        pos = np.argwhere(words == word)[0][0]
        tf_idf[enum] += tf[enumer] * idf[pos]

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [108]:
vect = TfidfVectorizer(stop_words=stop_list)

In [109]:
tf_idf_name = vect.fit_transform(data['name']).toarray().sum(axis = 1)

In [110]:
np.unique(tf_idf_name).__len__(), np.max(tf_idf_name), np.min(tf_idf_name)

(44101, 3.895829585422275, 0.0)

In [111]:
tf_idf_host_name = vect.fit_transform(data['host_name']).toarray().sum(axis=1)

In [112]:
np.unique(tf_idf_host_name).__len__(), np.max(tf_idf_host_name), np.min(tf_idf_host_name)

(1238, 2.2341552120887136, 0.0)

In [113]:
tf_idf_nhg = vect.fit_transform(data['neighbourhood_group']).toarray().sum(axis=1)

In [114]:
print(vect.get_feature_names())

['bronx', 'brooklyn', 'island', 'manhattan', 'queens', 'staten']


In [115]:
np.unique(tf_idf_nhg).__len__(), np.max(tf_idf_nhg), np.min(tf_idf_nhg)

(2, 1.414213562373095, 1.0)

In [116]:
tf_idf_ng = vect.fit_transform(data['neighbourhood']).toarray().sum(axis=1)

In [117]:
np.unique(tf_idf_ng).__len__(), np.max(tf_idf_ng), np.min(tf_idf_ng)

(117, 1.9031048479593613, 1.0)

In [118]:
tf_idf_rt = vect.fit_transform(data['room_type']).toarray().sum(axis=1)

In [119]:
np.unique(tf_idf_rt).__len__(), np.max(tf_idf_rt), np.min(tf_idf_rt)

(3, 1.7320508075688772, 1.2825880937756633)

In [120]:
tf_idf_lr = vect.fit_transform(data['last_review']).toarray().sum(axis=1)

In [121]:
np.unique(tf_idf_lr).__len__(), np.max(tf_idf_lr), np.min(tf_idf_lr)

(1498, 1.732037505564429, 1.0)

In [122]:
tfidf_test = pd.DataFrame.copy(data)

In [123]:
tfidf_test['name'] = tf_idf_name
tfidf_test['host_name'] = tf_idf_host_name
tfidf_test['neighbourhood_group'] = tf_idf_nhg
tfidf_test['neighbourhood'] = tf_idf_ng
tfidf_test['room_type'] = tf_idf_rt
tfidf_test['last_review'] = tf_idf_lr

In [124]:
tfidf_test.head(3)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,2.217081,2787,1.0,1.0,1.0,40.64749,-73.97237,1.414067,149,1,9,1.707679,0.21,6,365
1,2595,1.664189,2845,1.0,1.0,1.0,40.75362,-73.98377,1.732051,225,1,45,1.611607,0.38,2,355
2,3647,1.989233,4632,1.0,1.0,1.0,40.80902,-73.94190,1.414067,150,3,0,1.000000,0.00,1,365


In [125]:
price_col = np.array(tfidf_test['price'])
tfidf_test.drop(columns={'id', 'host_id', 'price'}, inplace=True)

In [126]:
tfidf_test = (tfidf_test - tfidf_test.mean()) / tfidf_test.std()

In [127]:
train_data, test_data, train_target, test_target = train_test_split(np.array(tfidf_test),
                                                                    price_col,
                                                                    test_size=0.3,
                                                                    random_state=241)

In [128]:
knn = KNNRegressor(8, "euclidean", "distance", "brute")

In [129]:
knn.fit(train_data, train_target)

In [130]:
res_tfidf = knn.predict(test_data)

In [131]:
RMSE(test_target, res_tfidf)

216.75665120209928

In [132]:
knn = KNNRegressor(8, "euclidean", "uniform", "brute")

In [133]:
knn.fit(train_data, train_target)

In [134]:
res_tfidf_uni = knn.predict(test_data)

(14669, 8)


In [135]:
RMSE(test_target, res_tfidf_uni)

215.04035397626836

In [601]:
knn = KNNRegressor(8, "cosine", "uniform", "brute")

In [602]:
knn.fit(train_data, train_target)

In [603]:
res_tfidf_cos = knn.predict(test_data)

(14669, 8)


In [604]:
RMSE(test_target, res_tfidf_cos)

198.61730453486683

In [532]:
only_text = tfidf_test[['name', 'host_name']]

In [534]:
train_data, test_data, train_target, test_target = train_test_split(np.array(only_text),
                                                                    price_col,
                                                                    test_size=0.3,
                                                                    random_state=241)

In [535]:
knn.fit(train_data, train_target)

In [536]:
res_text = knn.predict(test_data)

In [537]:
RMSE(test_target, res_text)

235.58439382558424

In [543]:
name_array = np.array(data['name'])
host_name_array = np.array(data['host_name'])

In [541]:
def modif_array(transf_array):
    for enum, item in enumerate(transf_array):
        transf_array[enum] = item.lower()
        transf_array[enum] = transf_array[enum].replace("  ", " ")
        transf_array[enum] = transf_array[enum].replace("/", " ")
        transf_array[enum] = transf_array[enum].replace("-", " ")
        transf_array[enum] = transf_array[enum].replace("...", " ")
        transf_array[enum] = transf_array[enum].replace("   ", " ")
        transf_array[enum] = transf_array[enum].replace("  ", " ")

    for enum in range(transf_array.__len__()):

        transf_array[enum] = mass_replace(transf_array[enum], ["!", ",", "&", ".", "'", ":", "*"])

In [544]:
modif_array(name_array)
modif_array(host_name_array)

In [546]:
name_dict = {}
name_split = []

for item in name_array:
    line = item.split(" ")
    line_kept = [x for x in line if x not in stop_list]
    name_split.append(line_kept)
    
    for part in line_kept:
        line_cur = []
        if part not in line_cur:
            line_cur.append(part)
            if part in name_dict:
                name_dict[part] += 1
            else:
                name_dict[part] = 1

In [550]:
host_name_dict = {}
host_name_split = []

for item in host_name_array:
    line = item.split(" ")
    line_kept = [x for x in line if x not in stop_list]
    host_name_split.append(line_kept)
    
    for part in line_kept:
        line_cur = []
        if part not in line_cur:
            line_cur.append(part)
            if part in host_name_dict:
                host_name_dict[part] += 1
            else:
                host_name_dict[part] = 1

In [551]:
name_sorted = [(k, name_dict[k]) for k in sorted(name_dict, key=name_dict.get, reverse=True)]
host_sorted = [(k, host_name_dict[k]) for k in sorted(host_name_dict, key=host_name_dict.get, reverse=True)]

In [559]:
name_sorted.__len__(), host_sorted.__len__()

(10903, 10137)

In [560]:
name_split.__len__(), host_name_split.__len__()

(48895, 48895)

In [558]:
j = 0
for item in name_sorted:
    if item[1] > 200:
        j += 1
        
j

192

In [582]:
def make_matrix(array, words_list, treshold):
    
    words = []
    count = []
    for item in words_list:
        words.append(item[0])
        count.append(item[1])
        
    words = np.array(words)
    count = np.array(count)
    
    cols = count[np.where(count > treshold)].__len__()
    
    result = np.zeros(array.__len__() * cols).reshape(array.__len__(), -1)
    
    for enum in range(array.__len__()):
        for item in array[enum]:
            pos = np.argwhere(words == item)[0][0]
            if pos < cols:
                result[enum][pos] += 1
    
    return result

In [717]:
name_matrix = make_matrix(name_split, name_sorted, 300)
host_matrix = make_matrix(host_name_split, host_sorted, 50)

In [718]:
data_matrix = np.c_[name_matrix, host_matrix]

In [719]:
price_col, data_matrix.shape

(array([149, 225, 150, ..., 115,  55,  90]), (48895, 311))

In [720]:
train_data, test_data, train_target, test_target = train_test_split(data_matrix,
                                                                    price_col,
                                                                    test_size=0.3,
                                                                    random_state=241)

In [721]:
train_data.shape

(34226, 311)

In [722]:
knn = KNNRegressor(8, "euclidean", "uniform", "brute")

In [723]:
knn.fit(train_data, train_target)

In [724]:
res_BOW = knn.predict(test_data)

(14669, 8)


In [725]:
RMSE(test_target, res_BOW)

220.338290854065

<b>3.2 (1 балл)</b> Используя все доступные признаки, решите задачу регрессии. Для категориальных и текстовых признаков выберите лучшие преобразования. Повлияло ли добавление количественного признака на метрику качества?

In [609]:
data.head(2)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,target
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,0
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,1


In [656]:
last_data = pd.DataFrame.copy(data)

In [19]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import normalize

In [20]:
encoder = LabelEncoder()

In [21]:
X = encoder.fit_transform(np.array(last_data['room_type']))

NameError: name 'last_data' is not defined

In [667]:
rt_cols = np.zeros((X.__len__(), 3))

In [677]:
for enum in range(X.__len__()):
    rt_cols[enum][X[enum]] = 2

In [675]:
Y = encoder.fit_transform(np.array(last_data['neighbourhood_group']))

In [676]:
np.unique(Y)

array([0, 1, 2, 3, 4])

In [671]:
nhg_cols = np.zeros((Y.__len__(), 5))

In [678]:
for enum in range(Y.__len__()): 
    nhg_cols[enum][Y[enum]] = 2

In [735]:
ng = ((tf_idf_ng - tf_idf_ng.mean()) / tf_idf_ng.std())[:, None]

In [736]:
lat = np.array(last_data['latitude'])
lat = ((lat - lat.mean()) / lat.std())[:, None]

In [738]:
lon = np.array(last_data['longitude'])
lon = ((lon - lon.mean()) / lon.std())[:, None]
lon.max(), lon.min()

(5.181953507723229, -6.331759967572333)

In [740]:
mn = np.array(last_data['minimum_nights'])
mn = ((mn - mn.mean()) / mn.std())[:, None]

In [741]:
data_matrix = np.c_[rt_cols, nhg_cols, name_matrix, host_matrix]

In [742]:
price_col

array([149, 225, 150, ..., 115,  55,  90])

In [743]:
train_data, test_data, train_target, test_target = train_test_split(data_matrix,
                                                                    price_col,
                                                                    test_size=0.3,
                                                                    random_state=241)

In [744]:
knn = KNNRegressor(9, "euclidean", "uniform", "brute")

In [745]:
knn.fit(train_data, train_target)

In [746]:
var1_res = knn.predict(test_data)

(14669, 9)


In [747]:
RMSE(test_target, var1_res)

214.82766671910215

## Часть 4: Выводы


In [39]:
# Ваши выводы здесь (ノ°∀°)ノ⌒･*:.｡. .｡.:*･゜ﾟ･*☆

In [41]:
tfidf_test.head(2)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,2.217081,2787,1.0,1.0,1.0,40.64749,-73.97237,1.414067,149,1,9,1.707679,0.21,6,365
1,2595,1.664189,2845,1.0,1.0,1.0,40.75362,-73.98377,1.732051,225,1,45,1.611607,0.38,2,355


In [43]:
tfidf_test.drop(columns={'id', 'host_id', 'latitude', 'longitude', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365'}, inplace=True)

In [45]:
tfidf_test.head(2)

,name,host_name,neighbourhood_group,neighbourhood,room_type,last_review
0,2.217081,1.0,1.0,1.0,1.414067,1.707679
1,1.664189,1.0,1.0,1.0,1.732051,1.611607


In [46]:
data_set = np.array(tfidf_test)
data_set[0]

array([2.21708089, 1.        , 1.        , 1.        , 1.41406694,
       1.70767877])

In [47]:
data_set = (data_set - data_set.mean()) / data_set.std()

In [49]:
price_col = np.array(data['price'])

In [50]:
train_data, test_data, train_target, test_target = train_test_split(data_set, price_col,
                                                                    test_size=0.3,
                                                                    random_state=241)

In [51]:
knn = KNNRegressor(75, 'euclidean', 'distance', 'brute')

In [52]:
knn.fit(train_data, train_target)

In [53]:
test_res = knn.predict(test_data)

In [54]:
RMSE(test_target, test_res)

211.66629190527252

In [323]:
x = np.array([[1, 2, 3, 4, 5], [6, 7, 3, 8, 5], [7, 1, 9, 10, 15]])
z = np.array([1, 2, 5, 8, 9])

In [328]:
knn = KNNRegressor(5, flattened_overlap, 'distance', 'brute')

In [329]:
knn.fit(x, np.arange(3))

In [330]:
knn.kneighbors(z)

(array([[1.73205081, 2.        , 2.23606798]]), array([[0, 1, 2]]))

range(0, 5)

In [291]:
testing = np.array(data['price'])

In [292]:
t = np.argsort(testing)

In [332]:
testing[t[-100:]]

array([ 2000,  2000,  2000,  2000,  2000,  2000,  2000,  2000,  2000,
        2000,  2000,  2000,  2000,  2000,  2010,  2100,  2103,  2200,
        2250,  2250,  2300,  2350,  2400,  2500,  2500,  2500,  2500,
        2500,  2500,  2500,  2500,  2500,  2500,  2500,  2545,  2590,
        2600,  2626,  2695,  2750,  2800,  2850,  2900,  2990,  2995,
        2999,  2999,  3000,  3000,  3000,  3000,  3000,  3000,  3000,
        3000,  3200,  3210,  3512,  3518,  3600,  3613,  3750,  3750,
        3800,  3900,  4000,  4000,  4000,  4100,  4160,  4200,  4200,
        4500,  4500,  5000,  5000,  5000,  5000,  5000,  5000,  5100,
        5250,  6000,  6000,  6419,  6500,  6500,  6500,  6800,  7500,
        7500,  7703,  8000,  8500,  9999,  9999,  9999, 10000, 10000,
       10000])

In [336]:
our_x = np.array(X_train[["neighbourhood_group", "neighbourhood", "room_type"]])
our_test = np.array(X_test[["neighbourhood_group", "neighbourhood", "room_type"]])

In [452]:
f_func = []
for item in our_x.T:
    uniques, counts = np.unique(item, return_counts=True)
    this = counts * (counts - 1) / (np.sum(counts)) / (np.sum(counts) - 1)
    f_func.append((this, uniques))


In [440]:
f_func[0]

(array([4.92442770e-04, 1.68772594e-01, 1.96679052e-01, 1.34447633e-02,
        5.61659987e-05]),
 array(['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island'],
       dtype=object))

In [462]:
def fo(y, z):
    """1 при различных, иначе какая-то дичь"""
    array = 0
    
    for enum in range(y.__len__()):
        if (y[enum] == z[enum]):
            f = f_func[enum][0]
            u = f_func[enum][1]
            pos = np.where(u == y[enum])[0][0]
            part_sum = (np.argwhere(f <= f[pos])).T[0]
            add = f[part_sum].sum()
            array += add
        else:
            array += 1

    return array

In [467]:
for i in range(1000):
    for j in range(10):
        print(fo(our_test[i], our_test[j]))

0.6660797004502832
2.479193450384816
1.661959416668228
1.661959416668228
2.479193450384816
2.479193450384816
2.479193450384816
3
2.479193450384816
2.479193450384816
2.479193450384816
0.8638206700385446
2.479193450384816
2.479193450384816
1.8586384688174855
2.479193450384816
1.8586384688174855
3
1.8586384688174855
0.8638206700385446
1.661959416668228
2.479193450384816
0.6929223542544536
1.661959416668228
2.479193450384816
2.479193450384816
2.479193450384816
3
2.479193450384816
2.479193450384816
1.661959416668228
2.479193450384816
1.661959416668228
0.6621197397251637
2.479193450384816
2.479193450384816
2.479193450384816
3
2.479193450384816
2.479193450384816
2.479193450384816
1.8586384688174855
2.479193450384816
2.479193450384816
0.8701759562623677
2.479193450384816
1.8586384688174855
3
1.8586384688174855
1.8586384688174855
2.479193450384816
2.479193450384816
2.479193450384816
2.479193450384816
2.479193450384816
0.4941202406706899
2.479193450384816
3
2.479193450384816
2.479193450384816
2.

2.2073256095125355
3
3
1.661959416668228
2.479193450384816
1.661959416668228
1.661959416668228
2.479193450384816
2.479193450384816
2.479193450384816
3
2.479193450384816
2.479193450384816
3
3
3
3
3
2.013993372033111
3
2.2073256095125355
3
3
2.1827659662834122
3
1.2137289038696377
2.1827659662834122
3
3
3
2.2073256095125355
3
3
2.479193450384816
1.8586384688174855
2.479193450384816
2.479193450384816
1.8586384688174855
2.479193450384816
1.8586384688174855
3
1.8586384688174855
1.8586384688174855
1.1868862500654673
3
2.1827659662834122
2.1827659662834122
3
3
3
2.2073256095125355
3
3
2.479193450384816
2.479193450384816
2.479193450384816
2.479193450384816
2.479193450384816
1.493186822417927
2.479193450384816
3
2.479193450384816
2.479193450384816
1.661959416668228
2.479193450384816
0.6929223542544536
1.661959416668228
2.479193450384816
2.479193450384816
2.479193450384816
3
2.479193450384816
2.479193450384816
2.479193450384816
2.479193450384816
2.479193450384816
2.479193450384816
2.479193450384

1.661959416668228
1.661959416668228
2.479193450384816
2.479193450384816
2.479193450384816
3
2.479193450384816
2.479193450384816
2.479193450384816
1.8586384688174855
2.479193450384816
2.479193450384816
1.8586384688174855
2.479193450384816
1.8586384688174855
3
1.8586384688174855
1.8586384688174855
2.1827659662834122
3
2.1827659662834122
2.1827659662834122
3
3
3
2.2073256095125355
3
3
3
2.3794450184326696
3
3
2.3794450184326696
3
2.3794450184326696
2.2073256095125355
2.3794450184326696
2.3794450184326696
1.661959416668228
2.479193450384816
1.661959416668228
1.661959416668228
2.479193450384816
2.479193450384816
2.479193450384816
3
2.479193450384816
2.479193450384816
1.1868862500654673
3
2.1827659662834122
2.1827659662834122
3
3
3
2.2073256095125355
3
3
3
3
3
3
3
1.014926790285874
3
2.2073256095125355
3
3
1.661959416668228
2.479193450384816
0.6929223542544536
1.661959416668228
2.479193450384816
2.479193450384816
2.479193450384816
3
2.479193450384816
2.479193450384816
1.661959416668228
2.479

2.1827659662834122
3
2.1827659662834122
2.1827659662834122
3
3
3
3
3
3
2.1827659662834122
3
2.1827659662834122
2.1827659662834122
3
3
3
2.2073256095125355
3
3
1.661959416668228
2.479193450384816
1.661959416668228
1.661959416668228
2.479193450384816
2.479193450384816
2.479193450384816
3
2.479193450384816
2.479193450384816
3
2.3794450184326696
3
3
2.3794450184326696
3
2.3794450184326696
2.2073256095125355
2.3794450184326696
2.3794450184326696
2.1827659662834122
3
2.1827659662834122
2.1827659662834122
3
3
3
2.2073256095125355
3
3
2.1827659662834122
3
2.1827659662834122
2.1827659662834122
3
3
3
3
3
3
3
2.3794450184326696
3
3
2.3794450184326696
3
2.3794450184326696
2.2073256095125355
2.3794450184326696
2.3794450184326696
2.479193450384816
1.8586384688174855
2.479193450384816
2.479193450384816
1.8586384688174855
2.479193450384816
1.8586384688174855
3
1.8586384688174855
1.8586384688174855
2.479193450384816
1.8586384688174855
2.479193450384816
2.479193450384816
1.8586384688174855
2.47919345038

2.1827659662834122
3
2.1827659662834122
2.1827659662834122
3
3
3
2.2073256095125355
3
3
2.479193450384816
1.8586384688174855
2.479193450384816
2.479193450384816
1.8586384688174855
2.479193450384816
1.8586384688174855
3
1.8586384688174855
1.8586384688174855
2.479193450384816
1.8586384688174855
2.479193450384816
2.479193450384816
1.8586384688174855
2.479193450384816
1.8586384688174855
3
1.8586384688174855
1.8586384688174855
2.479193450384816
1.8586384688174855
2.479193450384816
2.479193450384816
1.8586384688174855
2.479193450384816
1.8586384688174855
3
1.8586384688174855
1.8586384688174855
2.479193450384816
2.479193450384816
2.479193450384816
2.479193450384816
2.479193450384816
1.493186822417927
2.479193450384816
3
2.479193450384816
2.479193450384816
2.479193450384816
1.8586384688174855
2.479193450384816
2.479193450384816
1.8586384688174855
2.479193450384816
1.8586384688174855
3
0.8610370024847086
1.8586384688174855
3
2.3794450184326696
3
3
2.3794450184326696
3
2.3794450184326696
2.20732

2.479193450384816
1.8586384688174855
2.479193450384816
2.479193450384816
1.8586384688174855
2.479193450384816
1.8586384688174855
3
1.8586384688174855
1.8586384688174855
2.479193450384816
1.8586384688174855
2.479193450384816
2.479193450384816
1.8586384688174855
2.479193450384816
1.8586384688174855
3
1.8586384688174855
1.8586384688174855
2.479193450384816
0.8638206700385446
2.479193450384816
2.479193450384816
1.8586384688174855
2.479193450384816
1.8586384688174855
3
1.8586384688174855
0.8638206700385446
3
2.3794450184326696
3
3
1.3909825058775518
3
2.3794450184326696
2.2073256095125355
2.3794450184326696
2.3794450184326696
3
2.3794450184326696
3
3
2.3794450184326696
3
2.3794450184326696
2.2073256095125355
2.3794450184326696
2.3794450184326696
2.479193450384816
2.479193450384816
2.479193450384816
2.479193450384816
2.479193450384816
1.493186822417927
2.479193450384816
3
2.479193450384816
2.479193450384816
2.1827659662834122
3
2.1827659662834122
2.1827659662834122
3
3
3
2.2073256095125355
3

In [431]:
class flattened_overlap:
    
    def __init__(self):
        pass
    
    def fit(self, X_train):
        self.f = []
        self.l = X_train.shape[0]
        
        for column in X_train.T:
            d = dict(zip(*np.unique(column, return_counts=True)))
            self.f.append(d)
        
    def _f(self, j, x):
        if x in self.f[j].keys():
            return self.f[j][x]
        else:
            return 0
        
    def _p(self, j, x):
        return self._f(j, x) / self.l
    
    def _p_squared(self, j, x):
        return (self._f(j, x) * (self._f(j, x) - 1)) / (self.l * (self.l - 1))
    
    def __call__(self, x, z):
        result = 0.0
        for i in range(len(x)):
            if x[i] != z[i]:
                result += 1
            else:
                tresh = self._p(i, x[i])
                for q in list(self.f[i].keys()):
                    if self._p(i, q) <= tresh:
                        result += self._p_squared(i, q)
                        
        return result

In [432]:
fover = flattened_overlap()

In [433]:
fover.fit(our_x)

In [420]:
fover.l

34226

In [466]:
for i in range(1000):
    for j in range(10):
        print(fover(our_test[i], our_test[j]))

0.6660797004502831
2.479193450384816
1.661959416668228
1.661959416668228
2.479193450384816
2.479193450384816
2.479193450384816
3.0
2.479193450384816
2.479193450384816
2.479193450384816
0.8638206700385453
2.479193450384816
2.479193450384816
1.8586384688174855
2.479193450384816
1.8586384688174855
3.0
1.8586384688174855
0.8638206700385453
1.661959416668228
2.479193450384816
0.6929223542544535
1.661959416668228
2.479193450384816
2.479193450384816
2.479193450384816
3.0
2.479193450384816
2.479193450384816
1.661959416668228
2.479193450384816
1.661959416668228
0.6621197397251636
2.479193450384816
2.479193450384816
2.479193450384816
3.0
2.479193450384816
2.479193450384816
2.479193450384816
1.8586384688174855
2.479193450384816
2.479193450384816
0.8701759562623683
2.479193450384816
1.8586384688174855
3.0
1.8586384688174855
1.8586384688174855
2.479193450384816
2.479193450384816
2.479193450384816
2.479193450384816
2.479193450384816
0.49412024067068994
2.479193450384816
3.0
2.479193450384816
2.47919

2.479193450384816
2.479193450384816
2.479193450384816
1.493186822417927
2.479193450384816
3.0
2.479193450384816
2.479193450384816
1.661959416668228
2.479193450384816
0.6929223542544535
1.661959416668228
2.479193450384816
2.479193450384816
2.479193450384816
3.0
2.479193450384816
2.479193450384816
2.479193450384816
2.479193450384816
2.479193450384816
2.479193450384816
2.479193450384816
1.493186822417927
2.479193450384816
3.0
2.479193450384816
2.479193450384816
2.1827659662834122
3.0
1.2137289038696375
2.1827659662834122
3.0
3.0
3.0
2.2073256095125355
3.0
3.0
2.479193450384816
1.8586384688174855
2.479193450384816
2.479193450384816
1.8586384688174855
2.479193450384816
1.8586384688174855
3.0
1.8586384688174855
1.8586384688174855
2.479193450384816
2.479193450384816
2.479193450384816
2.479193450384816
2.479193450384816
1.493186822417927
2.479193450384816
3.0
2.479193450384816
2.479193450384816
3.0
2.3794450184326696
3.0
3.0
1.3909825058775525
3.0
2.3794450184326696
2.2073256095125355
2.379445

1.661959416668228
2.479193450384816
2.479193450384816
2.479193450384816
3.0
2.479193450384816
2.479193450384816
2.1827659662834122
3.0
2.1827659662834122
2.1827659662834122
3.0
3.0
3.0
3.0
3.0
3.0
2.1827659662834122
3.0
1.2137289038696375
2.1827659662834122
3.0
3.0
3.0
2.2073256095125355
3.0
3.0
3.0
3.0
3.0
3.0
3.0
2.013993372033111
3.0
2.2073256095125355
3.0
3.0
1.661959416668228
2.479193450384816
1.661959416668228
1.661959416668228
2.479193450384816
2.479193450384816
2.479193450384816
3.0
2.479193450384816
2.479193450384816
0.6660797004502831
2.479193450384816
1.661959416668228
1.661959416668228
2.479193450384816
2.479193450384816
2.479193450384816
3.0
2.479193450384816
2.479193450384816
3.0
3.0
3.0
3.0
3.0
2.013993372033111
3.0
2.2073256095125355
3.0
3.0
3.0
2.3794450184326696
3.0
3.0
2.3794450184326696
3.0
2.3794450184326696
2.2073256095125355
2.3794450184326696
2.3794450184326696
3.0
2.3794450184326696
3.0
3.0
2.3794450184326696
3.0
2.3794450184326696
2.2073256095125355
2.37944501

2.1827659662834122
3.0
2.1827659662834122
2.1827659662834122
3.0
3.0
3.0
2.2073256095125355
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
3.0
2.2073256095125355
3.0
3.0
2.479193450384816
1.8586384688174855
2.479193450384816
2.479193450384816
1.8586384688174855
2.479193450384816
1.8586384688174855
3.0
0.8610370024847094
1.8586384688174855
3.0
2.3794450184326696
3.0
3.0
2.3794450184326696
3.0
2.3794450184326696
2.2073256095125355
2.3794450184326696
2.3794450184326696
2.1827659662834122
3.0
2.1827659662834122
2.1827659662834122
3.0
3.0
3.0
2.2073256095125355
3.0
3.0
2.479193450384816
1.8586384688174855
2.479193450384816
2.479193450384816
1.8586384688174855
2.479193450384816
1.8586384688174855
3.0
1.8586384688174855
1.8586384688174855
1.661959416668228
2.479193450384816
1.661959416668228
1.661959416668228
2.479193450384816
2.479193450384816
2.479193450384816
3.0
2.479193450384816
2.479193450384816
1.661959416668228
2.479193450384816
0.6929223542544535
1.661959416668228
2.479193450384816
2.47919345038481

2.479193450384816
2.479193450384816
2.479193450384816
2.479193450384816
2.479193450384816
1.493186822417927
2.479193450384816
3.0
2.479193450384816
2.479193450384816
3.0
3.0
3.0
3.0
3.0
2.013993372033111
3.0
2.2073256095125355
3.0
3.0
2.1827659662834122
3.0
1.2137289038696375
2.1827659662834122
3.0
3.0
3.0
2.2073256095125355
3.0
3.0
2.479193450384816
1.8586384688174855
2.479193450384816
2.479193450384816
1.8586384688174855
2.479193450384816
1.8586384688174855
3.0
1.8586384688174855
1.8586384688174855
2.479193450384816
1.8586384688174855
2.479193450384816
2.479193450384816
1.8586384688174855
2.479193450384816
1.8586384688174855
3.0
1.8586384688174855
1.8586384688174855
3.0
2.3794450184326696
3.0
3.0
2.3794450184326696
3.0
2.3794450184326696
2.2073256095125355
2.3794450184326696
2.3794450184326696
1.661959416668228
2.479193450384816
1.661959416668228
1.661959416668228
2.479193450384816
2.479193450384816
2.479193450384816
3.0
2.479193450384816
2.479193450384816
2.479193450384816
1.8586384

In [ ]:
ты можешь вывести содержимое fit под  neighbourhood_group?

In [539]:
fover.f[0]

{'Bronx': 760,
 'Brooklyn': 14061,
 'Manhattan': 15179,
 'Queens': 3969,
 'Staten Island': 257}

In [445]:
f_func[0]

(array([4.92442770e-04, 1.68772594e-01, 1.96679052e-01, 1.34447633e-02,
        5.61659987e-05]),
 array(['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island'],
       dtype=object))

In [451]:
760 * 759 / 34226 / 34225

0.0004924427697694741

In [450]:
X_train.__len__()

34226

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
data.head(2)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355


In [13]:
vect = TfidfVectorizer(stop_words="english")

In [31]:
train_data, test_data, train_target, test_target = train_test_split(data.drop(columns={'price'}),
                                                                    np.array(data['price']),
                                                                    test_size=0.3,
                                                                    random_state=241)

In [32]:
train_data['name'] = vect.fit_transform(np.array(train_data['name'])).toarray().sum(axis=1)

In [33]:
test_data['name'] = vect.transform(np.array(test_data['name'])).toarray().sum(axis=1)

In [40]:
from sklearn.preprocessing import OneHotEncoder

In [41]:
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

In [43]:
X = encoder.fit_transform(np.array(train_data['host_id']).reshape(-1, 1))

In [44]:
Y = encoder.transform(np.array(test_data['host_id']).reshape(-1, 1))

In [49]:
Y.shape

(14669, 27425)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355


In [51]:
X1 = vect.fit_transform(np.array(train_data['host_id']).astype(str))

In [56]:
data.head(2)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355


In [141]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from math import sqrt

In [68]:
vect = TfidfVectorizer(stop_words='english')
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [347]:
train_data, test_data, train_target, test_target = train_test_split(pd.DataFrame.copy(data.drop(columns={'price'})),
                                                                    np.array(data['price']),
                                                                    test_size=0.3,
                                                                    random_state=241)

In [348]:
train_data['name'] = vect.fit_transform(np.array(train_data['name'])).toarray().sum(axis=1)
test_data['name'] = vect.transform(np.array(test_data['name'])).toarray().sum(axis=1)

In [349]:
train_data['host_name'] = vect.fit_transform(np.array(train_data['host_name'])).toarray().sum(axis=1)
test_data['host_name'] = vect.transform(np.array(test_data['host_name'])).toarray().sum(axis=1)

In [350]:
X = encoder.fit_transform(np.array(train_data['neighbourhood_group']).reshape(-1, 1)) * 1.2

In [351]:
train_data = train_data.join(pd.DataFrame(X, train_data.index))

In [352]:
Y = encoder.transform(np.array(test_data['neighbourhood_group']).reshape(-1, 1)) * 1.2

In [353]:
test_data = test_data.join(pd.DataFrame(Y, test_data.index))

In [354]:
X = encoder.fit_transform(np.array(train_data['room_type']).reshape(-1, 1)) * 1.2
Y = encoder.transform(np.array(test_data['room_type']).reshape(-1, 1)) * 1.2

In [355]:
train_data = train_data.join(pd.DataFrame(X, train_data.index, columns=range(5, 8)))
test_data = test_data.join(pd.DataFrame(Y, test_data.index, columns=range(5, 8)))

In [356]:
train_data.columns

Index([                            'id',                           'name',
                              'host_id',                      'host_name',
                  'neighbourhood_group',                  'neighbourhood',
                             'latitude',                      'longitude',
                            'room_type',                 'minimum_nights',
                    'number_of_reviews',                    'last_review',
                    'reviews_per_month', 'calculated_host_listings_count',
                     'availability_365',                                0,
                                      1,                                2,
                                      3,                                4,
                                      5,                                6,
                                      7],
      dtype='object')

In [357]:
train_data['neighbourhood'] = vect.fit_transform(np.array(train_data['neighbourhood'])).toarray().sum(axis=1)
test_data['neighbourhood'] = vect.transform(np.array(test_data['neighbourhood'])).toarray().sum(axis=1)

In [358]:
train_data.drop(columns={'id', 'neighbourhood_group', 'room_type'}, inplace=True)
test_data.drop(columns={'id', 'neighbourhood_group', 'room_type'}, inplace=True)

In [359]:
val = train_data['latitude'].mean()
std = train_data['latitude'].std()
train_data['latitude'] = (train_data['latitude'] - val) / std
test_data['latitude'] = (test_data['latitude'] - val) / std

val = train_data['longitude'].mean()
std = train_data['longitude'].std()
train_data['longitude'] = (train_data['longitude'] - val) / std
test_data['longitude'] = (train_data['longitude'] - val) / std

val = train_data['reviews_per_month'].mean()
std = train_data['reviews_per_month'].std()
train_data['reviews_per_month'] = (train_data['reviews_per_month'] - val) / std
test_data['reviews_per_month'] = (train_data['reviews_per_month'] - val) / std

val = train_data['number_of_reviews'].mean()
std = train_data['number_of_reviews'].std()
train_data['number_of_reviews'] = (train_data['number_of_reviews'] - val) / std
test_data['reviews_per_month'] = (train_data['reviews_per_month'] - val) / std

In [360]:
X = encoder.fit_transform(np.array(train_data['calculated_host_listings_count']).reshape(-1, 1))
Y = encoder.transform(np.array(test_data['calculated_host_listings_count']).reshape(-1, 1))

In [361]:
train_data = train_data.join(pd.DataFrame(X, train_data.index, range(8, 8+47)))
test_data = test_data.join(pd.DataFrame(Y, test_data.index, range(8, 8+47)))

In [362]:
train_data['last_review'] = vect.fit_transform(np.array(train_data['last_review'])).toarray().sum()
test_data['last_review'] = vect.transform(np.array(test_data['last_review'])).toarray().sum()

In [363]:
train_data.drop(columns={'calculated_host_listings_count', 'availability_365'}, inplace=True)
test_data.drop(columns={'calculated_host_listings_count', 'availability_365'}, inplace=True)

In [364]:
val = train_data['minimum_nights'].mean()
std = train_data['minimum_nights'].std()
train_data['minimum_nights'] = (train_data['minimum_nights'] - val) / std
test_data['minimum_nights'] = (test_data['minimum_nights'] - val) / std

In [365]:
train_data.drop(columns={'host_id'}, inplace=True)
test_data.drop(columns={'host_id'}, inplace=True)

In [366]:
knn = KNNRegressor(20, 'euclidean', 'distance', 'brute')

In [367]:
knn.fit(np.array(train_data), train_target)

In [368]:
attempt = knn.predict(np.array(test_data.fillna(0)))

In [369]:
RMSE(test_target, attempt)

214.38754000653185

In [370]:
train_data = (train_data - train_data.mean()) / (train_data.std())
test_data = (test_data - test_data.mean()) / (test_data.std())

In [371]:
knn.fit(np.array(train_data), train_target)

In [372]:
attempt_2 = knn.predict(np.array(test_data.fillna(0)))

In [373]:
RMSE(test_target, attempt_2)

208.63326405232598

In [376]:
new_data.isna().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [388]:
for enum, item in enumerate(np.array(new_data['last_review'])):
    if type(item) == str:
        if "2019-07-08" in item:
            print(item)

2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08
2019-07-08


In [843]:
new_data = pd.read_csv('AB_NYC_2019.csv')

In [844]:
# 1: id
new_data.drop(columns={'id'}, inplace=True)

In [702]:
# 2: name
#new_data.drop(columns={'name'}, inplace=True)

In [703]:
# 3: host_id
# можно предсказать mean, но нужно это делать отдельно на тестовом и тренировочном сетах

In [704]:
# 4: host_name
#new_data.drop(columns={'host_name'}, inplace=True)

In [579]:
# 6: nh
# предскажем mean

In [580]:
# 7: latitude + longitude

In [582]:
# 9: price

In [583]:
# 10: min_nights

In [584]:
# 11: numb_of_reviews

In [845]:
# 12: last_review
# самая поздняя дата - 2019.07.08
# чтобы не париться, пусть у нас в месяце 30 дней
st = '2019-07-08'
ye = int(st[:4])
mo = int(st[5:7])
day = int(st[8:])

In [846]:
for enum in range(new_data.__len__()):
    date = new_data['last_review'].loc[enum]
    if type(date) == str:
        value = 365 * (ye - int(date[:4])) + 30 * (mo - int(date[5:7])) + (day - int(date[8:]))
        new_data['last_review'].loc[enum] = value

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [847]:
new_data['last_review'].fillna(new_data['last_review'].mean(), inplace=True)

In [848]:
# 13: reviews_per_month
new_data['reviews_per_month'].fillna(0, inplace=True)

In [725]:
# 15: availability_365

In [710]:
new_data.head(2)

,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,264.0,0.21,6,365
1,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,47.0,0.38,2,355


In [817]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_target, test_target = train_test_split(new_data,
                                                                    np.array(new_data['price']),
                                                                    test_size=0.3,
                                                                    random_state=241)

In [727]:
def replaceToMeans(df, cols):
    for item in cols:
        helper = df.groupby(item, as_index=False)['price'].mean()
        comparer = np.array(helper[item])
        means = np.array(helper['price'])
        df[item+'new'] = 0
        for enum in df.index:
            pos = np.argwhere(comparer == df[item].loc[enum])[0][0]
            df[item+'new'].loc[enum] = means[pos]

In [728]:
def Transforming_fin(df, cols):
    
    for item in cols:
        
        helper = df.groupby(item)['price'].std()
        
        array = np.empty(df.__len__())
        
        for enum, element in enumerate(df[item]):
            
            array[enum] = helper[element]
            
        df[item+"new"] = array

def Transform_fin(train_df, test_df, cols):
    
    Transforming_fin(train_df, cols)

    for item in cols:
        
        replacement = np.empty(test_df.__len__())
        target_col = np.array(train_df[item])
        target_replacement = np.array(train_df[item+'new'])
        
        for enum, value in enumerate(test_df[item]):
            
            pos = np.argwhere(target_col == value).T[0]
            if pos.__len__() == 0:
                replacement[enum] = 0
            else:
                replacement[enum] = target_replacement[pos[0]]
                
        test_df[item] = replacement
        train_df[item] = train_df[item + 'new']
        train_df.drop(columns={item + 'new'}, inplace=True)

In [729]:
cols = ['neighbourhood_group', 'neighbourhood', 'room_type']
Transform_fin(train_data, test_data, cols)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [838]:
vect = TfidfVectorizer(stop_words='english')

In [839]:
train_data['name'].fillna("Noname", inplace = True)
test_data['name'].fillna("Noname", inplace = True)

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [840]:
train_data['name'] = vect.fit_transform(np.array(train_data['name'])).toarray().sum(axis=1)
test_data['name'] = vect.transform(np.array(test_data['name'])).toarray().sum(axis=1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [841]:
train_data['host_name'].fillna("Noname", inplace = True)
test_data['host_name'].fillna("Noname", inplace = True)

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:6287: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [842]:
train_data['host_name'] = vect.fit_transform(np.array(train_data['host_name'])).toarray().sum(axis=1)
test_data['host_name'] = vect.transform(np.array(test_data['host_name'])).toarray().sum(axis=1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [736]:
train_data.fillna(0, inplace=True)
test_data.fillna(0, inplace=True)

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:4244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


In [737]:
train_data.drop(columns={'price', 'host_id'}, inplace=True)

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [738]:
test_data.drop(columns={'price', 'host_id'}, inplace=True)

In [739]:
 train_data.head(2)

,name,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
39319,1.388041,1.0,205.538095,65.268367,40.63628,-73.9697,173.311647,2,12,1.0,3.67,3,220
22796,2.176624,1.0,290.655308,277.535740,40.73401,-73.9814,290.268598,2,15,570.0,0.58,1,0


In [751]:
cols = ['latitude', 'longitude', 'minimum_nights', 'number_of_reviews','calculated_host_listings_count', 'availability_365']
train_data.drop(columns=cols, inplace=True)
test_data.drop(columns=cols, inplace=True)

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [837]:
train_data.head(2)

,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
39319,Large bedroom,221940893,Sara,126.507523,93.514851,40.63628,-73.9697,90.613858,60,2,12,1.0,3.67,3,220
22796,Sun Drenched Gramercy/East Village!,68274298,Chelsea,198.168074,193.770492,40.73401,-73.9814,213.707274,226,2,15,570.0,0.58,1,0


In [753]:
knn = KNNRegressor(100, "euclidean", "uniform", "brute") # best - 7, 77 - Батенька, вы дебил

In [754]:
knn.fit(np.array(train_data), train_target)

In [755]:
result_euc_uni = knn.predict(np.array(test_data))

(14669, 100)


In [756]:
RMSE(test_target, result_euc_uni)

207.38703343639537

In [461]:
knn = KNNRegressor(30, "euclidean", "distance", "brute")

In [462]:
knn.fit(np.array(train_data), train_target)

In [463]:
result_euc_dis = knn.predict(np.array(test_data))

In [464]:
RMSE(test_target, result_euc_dis)

103.91941754959522

In [469]:
knn_cos = KNNRegressor(30, "cosine", "uniform", "brute")

In [474]:
knn_cos.fit(np.array(train_data), train_target)

In [479]:
result_cos_uni = knn.predict(np.array(test_data))

(14669, 25)


In [484]:
RMSE(test_target, result_cos_uni)

98.15000520818577

In [852]:
new_train_data = (train_data - train_data.mean()) / train_data.std()

In [853]:
new_test_data = (test_data - test_data.mean()) / test_data.std()

In [858]:
new_train_data.fillna(0, inplace=True)
new_test_data.fillna(0, inplace=True)

In [910]:
knn_normed = KNNRegressor(46, "euclidean", "uniform", "brute") # фолдинг, 50 - 195

In [911]:
knn_normed.fit(np.array(new_train_data), train_target)

In [912]:
res_normed = knn_normed.predict(np.array(new_test_data))

(14669, 46)


In [913]:
RMSE(test_target, res_normed), mean_absolute_error(test_target, res_normed)

(195.5156164054704, 62.59969115585366)

In [770]:
data.loc[train_data.index[[1, 2, 4, 6]]]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
22796,18458323,Sun Drenched Gramercy/East Village!,68274298,Chelsea,Manhattan,Gramercy,40.73401,-73.98140,Entire home/apt,226,2,15,2017-12-18,0.58,1,0
26190,20886432,Gorgeous single room in Sunnyside near Manhattan,75458625,Lidia,Queens,Sunnyside,40.74722,-73.91860,Private room,78,2,20,2019-07-01,0.91,4,365
32473,25452071,Modern & Cozy Studio In Best East Village Loca...,3142924,Kea,Manhattan,East Village,40.72714,-73.98215,Entire home/apt,190,2,32,2019-07-06,2.74,1,17
25954,20728525,Lovely Contemporary Home away from Home,141052003,Nicole,Queens,Jamaica,40.68503,-73.77983,Entire home/apt,150,2,45,2019-06-30,2.01,4,153


In [834]:
def kfold(n, n_folds):

    index_list = np.arange(n)

    result_list = []
    part = 0
    each_len = int(n / n_folds)

    while part < n_folds - 1:
        test_subset = np.array(index_list[part * each_len: (part + 1) * each_len])
        train_subset = np.array([x for x in index_list if x not in test_subset])
        result_list.append((train_subset, test_subset))
        part += 1

    test_subset = np.array(index_list[part * each_len:])
    train_subset = np.array([x for x in index_list if x not in test_subset])

    result_list.append((train_subset, test_subset))

    return result_list

def Transforming_fold_like(train_df, test_df, cols):
            
    prices = np.array(train_df['price'])
    cv = kfold(train_df.__len__(), 5)
    
    for item in cols:
        
        array = np.empty(train_df.__len__())
        
        target_col = np.array(train_df[item])

        for fold in cv:
            
            train = target_col[fold[0]]
            test = target_col[fold[1]]
            
            for value in np.unique(train):
                subset_pos = np.argwhere(train == value).T[0]
                real_pos = fold[0][subset_pos]
                this_mean = prices[real_pos].mean()
                
                change_pos = fold[1][np.argwhere(test == value).T[0]]
                array[change_pos] = this_mean
              
        allocations = np.array(train_df[item])
        train_df[item] = array
        
        test_array = np.empty(test_df.__len__())
        test_values = np.array(test_df[item])
        
        for value in np.unique(test_values):
            pos = np.argwhere(test_values == value).T[0]
            alloc_pos = np.argwhere(allocations == value).T[0]
            test_array[pos] = array[alloc_pos].mean()
            
        test_df[item] = test_array
            

In [849]:
train_data, test_data, train_target, test_target = train_test_split(new_data,
                                                                    np.array(new_data['price']),
                                                                    test_size=0.3,
                                                                    random_state=241)

In [850]:
cols = ['host_id', 'neighbourhood_group', 'neighbourhood', 'room_type']
Transforming_fold_like(train_data, test_data, cols)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:55: RuntimeWarning: Mean of empty slice.
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [851]:
train_data.drop(columns={'price', 'name', 'host_name'}, inplace=True)
test_data.drop(columns={'price', 'name', 'host_name'}, inplace=True)

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
